In [ ]:
# %pip install --upgrade "mlflow[databricks]"
# %pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# # %pip install pycaret
# # %pip install catboost
# # %pip install lightgbm

# %pip install xgboost
# %pip install scikit-optimize
# #%pip install fastapi
# #%pip install uvicorn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Training XGBoost

In [ ]:
import os
import mlflow
import mlflow.xgboost

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer
#from dotenv import load_dotenv

In [ ]:
EXPERIMENT_NAME = os.getenv("Kaggle_GiveCredit_Experiment", "/Users/2407.adi@gmail.com/Kaggle_GiveCredit_Experiment")
REGISTERED_MODEL_NAME = os.getenv("MLFLOW_REGISTERED_MODEL", "adityaspersonal.default.CreditRiskModel")
SEED = int(os.getenv("SEED", "42"))

# paths (for local dev or DBFS reads)
TRAIN_PATH = os.getenv("TRAIN_PATH", "/data/cs-training.csv")
TARGET_COL = os.getenv("TARGET_COL", "SeriousDlqin2yrs")

In [ ]:
import os

#load_dotenv()
DATABRICKS_HOST = os.getenv("DATABRICKS_HOST")
DATABRICKS_TOKEN = os.getenv("DATABRICKS_TOKEN")

import mlflow
mlflow.set_registry_uri("databricks-uc")
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/4241506986123439', creation_time=1755684055342, experiment_id='4241506986123439', last_update_time=1756979605451, lifecycle_stage='active', name='/Users/2407.adi@gmail.com/Kaggle_GiveCredit_Experiment', tags={'mlflow.databricks.filesystem.experiment_permissions_check': 'test',
 'mlflow.experiment.sourceName': '/Users/2407.adi@gmail.com/Kaggle_GiveCredit_Experiment',
 'mlflow.experimentKind': 'custom_model_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': '2407.adi@gmail.com',
 'mlflow.ownerId': '140870377873547'}>

In [23]:
from utils import prob_to_log_odds, log_odds_to_score, to_2d_frame, build_search_space, load_training_data
from preprocess import preprocess_input

In [24]:
# Load data
raw = load_training_data(TRAIN_PATH)
raw.columns = ['Unnamed: 0', 'SeriousDlqin2yrs',
       'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30_59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60_89DaysPastDueNotWorse',
       'NumberOfDependents']
X_raw = raw.drop(columns=[TARGET_COL, 'Unnamed: 0'])
y = raw[TARGET_COL].astype(int)

In [26]:
# Ensure all numeric fields are float (so they allow NaN)
X_raw = X_raw.astype({col: 'float' for col in X_raw.columns})

In [27]:
# Preprocess
X_raw_1 = X_raw.copy()
X_raw = preprocess_input(X_raw)

In [30]:
# Basic split 75/25
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.25, stratify=y, random_state=SEED)

In [31]:
X_raw_1 = X_raw_1.iloc[0:X_train.shape[0], :]
print(X_raw_1.shape, X_train.shape)

(112500, 10) (112500, 15)


In [32]:
# Base estimator
base = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    random_state=SEED,
    n_jobs=-1
)

In [41]:
# Bayesian optimization
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
opt = BayesSearchCV(
    estimator=base,
    search_spaces=build_search_space(),
    scoring="roc_auc",
    cv=cv,
    n_iter=int(os.getenv("BAYES_N_ITER", "100")),
    n_jobs=-1,
    verbose=1,
    random_state=SEED,
    refit=True)

In [42]:
import os
from mlflow.models.signature import infer_signature
import mlflow
import numpy as np
from sklearn.metrics import roc_auc_score

with mlflow.start_run() as run:
    # 🔹 Fit Bayes optimizer
    opt.fit(X_train, y_train)

    # 🔹 Best model and params
    best_model = opt.best_estimator_
    best_params = opt.best_params_
    best_cv_auc = opt.best_score_

    # 🔹 Evaluate on held-out test
    test_prob = best_model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, test_prob)

    # 🔹 Log metrics and hyperparams
    mlflow.log_metric("cv_auc", best_cv_auc)
    mlflow.log_metric("test_auc", test_auc)
    mlflow.log_params(best_params)

    # # 🔹 Infer input/output signature
    signature = infer_signature(X_raw_1, best_model.predict_proba(X_train))

    # ================================================================
    # 1. Log raw XGBoost model (artifact only, NOT registered)
    # ================================================================
    mlflow.xgboost.log_model(
        xgb_model=best_model,
        artifact_path="xgb_model",
        signature=signature
    )

    # ================================================================
    # 2. Log + Register PyFunc wrapper (deployment-ready model)
    # ================================================================
    from serve_pyfunc import CreditRiskPyFunc
    import mlflow.pyfunc

    input_example = X_raw_1.head(1)
    sig_cols = X_raw_1.columns.to_list()

    mlflow.pyfunc.log_model(
        artifact_path="pyfunc_model",
        python_model=CreditRiskPyFunc(),
        artifacts={
            "xgb_model_uri": f"runs:/{run.info.run_id}/xgb_model"  # points to raw booster above
        },
        input_example=input_example[sig_cols],
        signature=signature,
        registered_model_name=REGISTERED_MODEL_NAME  # ✅ only this goes to registry
    )

    # 🔹 Print run info
    print(f"Run ID: {run.info.run_id}")
    print(f"Registered Model: {REGISTERED_MODEL_NAME}")
    print(f"Best CV AUC: {best_cv_auc:.4f} | Test AUC: {test_auc:.4f}")

    # 🔹 Save run info for next step in pipeline
    current_run_id = run.info.run_id
    current_test_auc = test_auc
    current_model_uri = f"runs:/{run.info.run_id}/xgb_model"

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

2025/09/04 15:30:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
🔗 View Logged Model at: https://adb-4193764046904389.9.azuredatabricks.net/ml/experiments/4241506986123439/models/m-b19c341546584ebc9b1e23391bd63b41
/Users/macbookpro/Documents/.venv/lib/python3.12/site-packages/xgboost/sklearn.py:1028: UserWarning: [15:30:13] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/09/04 15:30:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
🔗 View Logged Model at: https://adb-4193764046904389.9.azuredatabricks.net/ml/experiments/4241506986123439/models/m-30bb286c7efd452eb55a6aa8cc22b33a
2025/09/04 15:30:28 INFO mlflow.pyfunc: Validating input example against model signature
Registered model 'adityaspersonal.default.Credi

Run ID: 2b118dc099a0486bb5dab91ad2c3965a
Registered Model: adityaspersonal.default.CreditRiskModel
Best CV AUC: 0.8650 | Test AUC: 0.8690
🏃 View run lyrical-sow-980 at: https://adb-4193764046904389.9.azuredatabricks.net/ml/experiments/4241506986123439/runs/2b118dc099a0486bb5dab91ad2c3965a
🧪 View experiment at: https://adb-4193764046904389.9.azuredatabricks.net/ml/experiments/4241506986123439


In [ ]:
from mlflow.tracking import MlflowClient
import numpy as np

client = MlflowClient()

IMPROVEMENT_DELTA = 0  # Set your threshold here

model_promoted = False # tells my CICD Pipeline if model was promoted

# 1. Find the Production version (if it exists)
prod_versions = client.search_model_versions(f"name='{REGISTERED_MODEL_NAME}'")
prod_version = None
prod_run_id = None

for v in prod_versions:
    mv = client.get_model_version(REGISTERED_MODEL_NAME, v.version)
    if "production" in mv.aliases:
        prod_version = int(v.version)
        prod_run_id = v.run_id
        print(prod_version, prod_run_id)
        break

# 2. Get the Production model's test_auc (if exists)
prod_auc = float("-inf")
if prod_version is not None:
    prod_run = client.get_run(prod_run_id)
    prod_auc = float(prod_run.data.metrics.get("test_auc", "nan"))

print(f"Current test_auc: {current_test_auc}, Production test_auc: {prod_auc}")

# 3. Compare and decide
if (prod_version is None) or (
    not np.isnan(current_test_auc) 
    and current_test_auc >= prod_auc + IMPROVEMENT_DELTA):
    model_promoted = True # tells my CICD Pipeline if model was promoted
    print("✅ Current model is better (or no prod exists). Promoting to Production...")

    # Find the latest version created for this run
    # (PyFunc was registered in your training code)
    versions = client.search_model_versions(f"name='{REGISTERED_MODEL_NAME}'")
    current_version = None
    for v in versions:
        if v.run_id == current_run_id:
            current_version = int(v.version)
            break

    if current_version is None:
        raise RuntimeError("No registered version found for current run!")

    # Transition current version to Production
    client.set_registered_model_alias(
        name=REGISTERED_MODEL_NAME,
        version=current_version,
        alias="production")

    print(f"Promoted version {current_version} of {REGISTERED_MODEL_NAME} to Production.")

else:
    print("🚫 Production model is better. Doing nothing.")

9 1e97e124f1924bfbbe8dc9ddb1ba5839
Current test_auc: 0.8689630056749895, Production test_auc: 0.8694149130387919
✅ Current model is better (or no prod exists). Promoting to Production...
Promoted version 11 of adityaspersonal.default.CreditRiskModel to Production.


In [ ]:
with open("model_promotion_result.txt", "w") as f:
    f.write("PROMOTED" if model_promoted else "NO_PROMOTION")

In [ ]:
# train_preds = best_model.predict_proba(X_raw)[:, 1]
# df_preds = pd.DataFrame({
#     "score": train_preds,
#     "label": y
# }, index=y.index)
# # Save predictions
# df_preds.to_parquet("data/train_predictions.parquet")

# Getting SHAP values

In [ ]:
# import shap
# explainer = shap.TreeExplainer(best_model)

In [ ]:
# shap_values = explainer.shap_values(X_train)

In [ ]:
# import pandas as pd

# feat_importance = pd.DataFrame(
#     best_model.get_booster().get_score(importance_type="gain").items(),
#     columns=["feature", "importance"]
# ).sort_values(by="importance", ascending=False)

In [ ]:
# feat_importance

,feature,importance
12,SeriousDelinqRate,969.122314
0,RevolvingUtilizationOfUnsecuredLines,569.151428
2,NumberOfTime30_59DaysPastDueNotWorse,526.042664
6,NumberOfTimes90DaysLate,332.212799
10,CreditUtilizationPerLine,310.020447
8,NumberOfTime60_89DaysPastDueNotWorse,257.369629
7,NumberRealEstateLoansOrLines,80.062454
14,AgePerCreditLine,64.475868
1,age,60.742039
5,NumberOfOpenCreditLinesAndLoans,54.313786


In [ ]:
# feat_importance.to_parquet("data/feature_importance.parquet")

# Testing API Locally

In [ ]:
# import requests

# sample = {
#   "RevolvingUtilizationOfUnsecuredLines": 0.35,
#   "age": 45,
#   "NumberOfTime30_59DaysPastDueNotWorse": 1,
#   "DebtRatio": 0.65,
#   "MonthlyIncome": 5500,
#   "NumberOfOpenCreditLinesAndLoans": 8,
#   "NumberOfTimes90DaysLate": 0,
#   "NumberRealEstateLoansOrLines": 2,
#   "NumberOfTime60_89DaysPastDueNotWorse": 0,
#   "NumberOfDependents": 2.0
# }

# resp = requests.post("http://127.0.0.1:8000/predict_1", json=sample)
# print(resp.status_code)   # should be 200
# print(resp.text)          # raw response
# print(resp.json())        # parse JSON only if status_code == 200

200
{"score":611.8424072265625,"percentile":77.0,"explanations":["Your credit score is positively impacted by 30-59 Days Past Due, Debt Ratio.","Your credit score is negatively impacted by SeriousDelinqRate, Revolving Utilization."],"features":{"RevolvingUtilizationOfUnsecuredLines":0.35,"age":45.0,"NumberOfTime30_59DaysPastDueNotWorse":1.0,"DebtRatio":0.65,"MonthlyIncome":5500.0,"NumberOfOpenCreditLinesAndLoans":8.0,"NumberOfTimes90DaysLate":0.0,"NumberRealEstateLoansOrLines":2.0,"NumberOfTime60_89DaysPastDueNotWorse":0.0,"NumberOfDependents":2.0,"TotalObligation":3575.0,"TotalUnsecuredLoans":4.0,"TotalSecuredLoans":4.0,"CreditCards":1.0,"PersonalLoans":1.0,"BNPLLoans":1.0,"OtherUnsecured":1.0,"RealEstateLoans":2.0,"GoldLoans":0.0,"VehicleLoans":1.0,"OtherSecured":1.0,"HasDelinquencyHistory":1.0,"DependentsFlag":1.0,"TotalLoanUtilization":0.52,"BureauVintage":12.0,"NumberOfEnquiriesInLast6Months":24.0},"force_plot":"..."}
{'score': 611.8424072265625, 'percentile': 77.0, 'explanations'

In [ ]:
# import requests

# sample = {"customer_id":2}

# resp = requests.post("http://localhost:8000/predict", json=sample)
# print(resp.status_code)   # should be 200
# #print(resp.text)          # raw response
# print(resp.json())        # parse JSON only if status_code == 200

200
{'score': 597.8682250976562, 'percentile': 71.97, 'explanations': ['Your credit score is positively impacted by Number of Real Estate Loans, AgePerCreditLine.', 'Your credit score is negatively impacted by 30-59 Days Past Due, SeriousDelinqRate.'], 'features': {'RevolvingUtilizationOfUnsecuredLines': 0.46, 'age': 57.0, 'NumberOfTime30_59DaysPastDueNotWorse': 0.0, 'DebtRatio': 0.53, 'MonthlyIncome': 9141.0, 'NumberOfOpenCreditLinesAndLoans': 15.0, 'NumberOfTimes90DaysLate': 0.0, 'NumberRealEstateLoansOrLines': 4.0, 'NumberOfTime60_89DaysPastDueNotWorse': 0.0, 'NumberOfDependents': 2.0, 'TotalObligation': 4819.0, 'TotalUnsecuredLoans': 10.0, 'TotalSecuredLoans': 5.0, 'CreditCards': 5.0, 'PersonalLoans': 1.0, 'BNPLLoans': 2.0, 'OtherUnsecured': 2.0, 'RealEstateLoans': 2.0, 'GoldLoans': 1.0, 'VehicleLoans': 1.0, 'OtherSecured': 1.0, 'HasDelinquencyHistory': 0.0, 'DependentsFlag': 1.0, 'TotalLoanUtilization': 0.4, 'BureauVintage': 7.0, 'NumberOfEnquiriesInLast6Months': 39.0}, 'force_plo